In [ ]:
!pip install japanize-matplotlib

In [ ]:
from google.colab import drive
drive.mount('gdrive')
#drive.flush_and_unmount()

In [ ]:
path = '/content/gdrive/My Drive/感染症シミュレーション（定量生物学）/ver1'
savetitle_header = 'Virus_Infection_Network_SIR_ver1'
repeat_indices = range(10)
parms = [0.5, 0.2]
parms_str = 'reduction1'

taskID_range = list(range(1,len(parms)+1))

In [ ]:
# coding: utf-8

###############################################################
##########    Infection propagation simulator   ###############
##########    Original author: Yuichi Iino      ###############
###############################################################

###  Please refer to the powerpoint slides for the principle of simulation ####

# use lognormal distribution in Network_SEIR_3_3
# random connection in 3_4
# Make movie in 4_1
# Consider geometrical distance for infection probability in 4_2
# Introduce travel restriction in 4_3
# Modify lognormal according to 4_1_1 in 4_4
# Do not use infection probability matrix but istead use list to possibly allow for a large N in 5_1
# Initialize for each repeat for n_repeat, and repeat in the same condiitons for repaet in n_sub_repeat in 5_2
# save data in 5_3
# version 6 is free of scale. Total number of people N is variable and means the total number of E or I or R people.
# variable Status is np.int8 showing the infection status. 0 is Susceptible, 1 is Exposed, 2 is Infectious, 3 is Recovered
# x and y; mean position of each person.
# in version 7_1, wandering radius is introduced.
# in version 7_2, high risk population is omitted.
# in version 8_1, new type of virus emerges.
# in version 9_1, we can start with multiple infectants (revival of initial_E_number). Average rectangular distance between individuals is set to rmean*2.
# in SIR version 10_1, major change. ws is introduced. E is omitted and new infection is calculated.

import numpy as np
import math
from matplotlib import pyplot as plt
import japanize_matplotlib
from scipy import stats
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from matplotlib import animation as anm
import os
import random


############## Preset Parameters ################

max_n = 100000 # number of maximal number of infected people, where simulation stops.
b=0.2 # Exposed -> Infectious 速度 (not currently used)
c=0.1 # Infections -> Recovery 速度 (not currently used)
t_end=210 # シミュレーション日数
t_new=60 # Type2ウイルス導入時
growth_rate_factor = 1.48
injected_trials = range(10) # [] for none, -1 for all.
initial_E_number = 5 # 最初の潜伏感染者数
initial_E_spacing = 30
n_repeat = 100 # number of repeats
n_sub_repeat = 1 # number of repeats in same conditions (always=1)
show_daily_new_infectants = True
save_data = True
ws_length = 20
reduction1=0.5 # 5割自粛
reduction2=1 # 解除
t1=120 # 自粛開始日
t2=180 # 解除日

serial_interval_type = 'Bi' # 'Nishiura'

sigma1 = 0.001; mean1 = 1.16 # first distribution of R0
ksigma=0.2; kmean=15 # distribution of k (number of people to contact)
rsigma=1; rmean=15 # distribution of r (radius of movment of the person)
max_daily = 50 # y axis max for daily number of new infectants
max_dailysum = 10000 # y axis max for summed daily number of new infectants (0 for auto)

############## parameters for animation ###################

img_w   = 12 # 画像サイズ
img_h   = 12 # 画像サイズ
resolution = 100 #25
interval = 200
frames = t_end

a_max = 2# 0.5
title_length = 6
settle_length = 10

min_area_halfsize = 16
max_area_halfsize = 64
show_n_repeats =  10
# will be devided by n0
labeledgesize = 10000
labelsize = 5000 #4000
headwidth = 6 #4 #3
headlength = 4 #3
linewidth = 30
arrowwidth = 2 #1

In [ ]:
#########################################
#########    Functions    ###############
#########################################

################################
#確率分布
################################

# log normal distribution for both a and k

def prob_R0():
    R0 = np.random.lognormal(mean=math.log(scale1), sigma=sigma1, size=1)
    return(R0[0])

def prob_a(s,R0):
    return(ws[s]*R0)

def prob_k():
    k = np.random.lognormal(mean=math.log(kscale), sigma=ksigma, size=1).astype(int)
    return(k[0])

def prob_r():
    r = np.random.lognormal(mean=math.log(rscale), sigma=rsigma, size=1)
    return(r[0])

######################  感染伝播  ####################

def spread(t, R0_vec, Status, Infected, NewInfectants, InfectedFrom, Type, X, Y, sum_a_vec, k_vec, N, repeat_i1):

  # SIR モデルに従いｔ-1までにおけるStatusを受け取りtにおけるStatusを返す
  #print('t=',t)
  old_N = N
  Status[:,t] = Status[:,t-1]

  ####  progression of status  ####
  # S=0, E=1, I=2, R=3,  in this version E is absent.

  # determine transition I -> R
  if t-ws_length>=0:
    recovered = np.where(Infected[:,t-ws_length])[0]
    Status[recovered, t] = 3

  ### new virus
  if t==t_new and (np.any(np.array(injected_trials)<0) or np.any(repeat_i1==np.array(injected_trials))):
      Type = np.append(Type, 1)
      x = int( np.min(X) + (np.max(X) - np.min(X)) * random.random() )
      y = int( np.min(Y) + (np.max(Y) - np.min(Y)) * random.random() )
      while len(np.where(np.logical_or(X!=x, Y!=y))[0])==0:
        x = x + int(random.random()*10)
        y = y + int(random.random()*10)
      Status = np.concatenate((Status, np.zeros((1,t_end), dtype=np.int8)))
      N = N + 1
      Status[N-1,t] = 2
      Infected = np.concatenate((Infected, np.zeros((1,t_end), dtype=np.int8)))
      Infected[N-1,t] = 1
      InfectedFrom = np.append(InfectedFrom, -1)
      X = np.append(X,x)
      Y = np.append(Y,y)
      R0_vec = np.append(R0_vec, prob_R0())
      sum_a_vec = np.append(sum_a_vec, -1)
      k = prob_k()
      k_vec = np.append(k_vec, k)
  else:

  ###   Infection   ###

    for s in range(1,(min(ws_length,t)+1)):
      ts = t-s
      infectious = np.where(Infected[:,ts]>0)[0]
      if len(infectious) > 0: # if there is any infections people
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        for i in infectious: # for each infectious person
            # Determine a for infectious person i
            a = prob_a(s,R0_vec[i])

            # new type of virus is more infectious
            if Type[i]>0:
              a = a * growth_rate_factor
            if t<t1:
              a = a
            elif t<t2:
              a = a*reduction1
            else:
              a = a*reduction2

            sum_a_vec[i] += a
            # Determine k for infections person i
            k = prob_k()
            if k > 0:
              for ki in range(k):
              # Draw and determine whether i infects (at probability a/k)
                if random.random() <= a/k:
                  
                ###### infection occurs #######

                  # determine distance
                  max_r = prob_r()
                  # 距離に比例した確率で割り振る（確率は円周に比例）
                  r = np.sqrt(random.random())*max_r
                  # determine angle
                  angle = 2*np.pi*random.random()
                  # determine position of target
                  dx = int(r*math.cos(angle))
                  dy = int(r*math.sin(angle))
                  if dx==0 and dy==0:
                    dx = np.sign(math.cos(angle)) * int(abs(math.cos(angle))>=math.sqrt(0.5))
                    dy = np.sign(math.sin(angle)) * int(abs(math.sin(angle))>=math.sqrt(0.5))
                  # see whether target is already infected
                  x = X[i] + dx
                  y = Y[i] + dy
                  target = np.where(np.logical_and(X==x, Y==y))[0]
                  if len(target)==0 and N<max_n:
                    # if target candidate is uninfected
                    if N==max_n-1:
                      print('Reached '+str(max_n)+' infectants and stop here.')
                    Status = np.concatenate((Status, np.zeros((1,t_end), dtype=np.int8)))
                    Infected = np.concatenate((Infected, np.zeros((1,t_end), dtype=np.int8)))
                    N = N+1
                    # infects. N -> N+1
                    Status[N-1,t] = 2 # becomes I
                    Infected[N-1,t] = 1

                    # record InfectedFrom
                    InfectedFrom = np.append(InfectedFrom, i)

                    Type = np.append(Type, Type[i])
                    # record new position
                    X = np.append(X,x)
                    Y = np.append(Y,y)
                    sum_a_vec = np.append(sum_a_vec, a)
                    k_vec = np.append(k_vec, k)
                    R0_vec = np.append(R0_vec, prob_R0())

  # record NewInfectants (int vector)
  NewInfectants[t,0] = np.sum(np.logical_and(Infected[:,t]==1, Type==0))
  NewInfectants[t,1] = np.sum(np.logical_and(Infected[:,t]==1, Type==1))

  return(R0_vec, Status, Infected, NewInfectants, InfectedFrom, Type, X, Y, sum_a_vec, k_vec, N)


###################### 時間進行実施  ####################

def simulate():

    # 繰り返しのシミュレーションを実施
    
    repeat_i = 0
    cumStatus = []
    cumInfected = []
    cumInfectedFrom = []
    cumNewInfectants = np.zeros((n_repeat*n_sub_repeat, t_end, 2),dtype=int)
    cum_sum_a_vec = []
    cum_k_vec = []
    cum_R0_vec = []
    cumX = []
    cumY = []
    cumN = []
    cumType = []
    
    for repeat_i1 in range(n_repeat):
        
        #print("repeat" + str(repeat_i1))
        print(str(repeat_i1), end=' ')
        if (repeat_i1+1)%40 == 0 or repeat_i1+1==n_repeat:
          print('')

        #######################
        #シミュレーション準備
        #######################

        #初期状態

        Status= np.zeros((initial_E_number, t_end), dtype=np.int8)
        Status[:,0] = 2 # infected # Exposed
        Infected = np.zeros((initial_E_number, t_end), dtype=np.int8)
        Infected[:,0] = 1
        X = np.array([], dtype=int)
        Y = np.array([], dtype=int)
        for i in range(initial_E_number):
          x = int((random.random()*2-1)/2*initial_E_spacing*math.sqrt(initial_E_number))
          y = int((random.random()*2-1)/2*initial_E_spacing*math.sqrt(initial_E_number))
          X = np.append(X, x)
          Y = np.append(Y, y)
        InfectedFrom = np.array([-1]*initial_E_number, dtype=int)
        NewInfectants = np.array(np.zeros((t_end,2), dtype=int)) # number of new infectants, first column=old virus, second column=new virus.
        Type = np.array([0]*initial_E_number, dtype = int)
        times = np.arange(t_end)
        sum_a_vec = np.array([0.0]*initial_E_number)
        k_vec = np.array([-1]*initial_E_number)
        R0_vec = np.array([-1.0]*initial_E_number)
        for i in range(initial_E_number):
          R0_vec[i] = prob_R0()
        N = initial_E_number
        
        ############################
        # シミュレーションリピート
        ############################

        for repeat_i2 in range(n_sub_repeat):

            #print("subrepeat" + str(repeat_i2))
            
            for t in times[1:-1]:
              R0_vec, Status, Infected, NewInfectants, InfectedFrom, Type, X, Y, sum_a_vec, k_vec, N =\
                spread(t, R0_vec, Status, Infected, NewInfectants, InfectedFrom, Type, X, Y, sum_a_vec, k_vec, N, repeat_i1)
            cumStatus.append(Status)
            cumInfected.append(Infected)
            cumInfectedFrom.append(InfectedFrom)
            cumNewInfectants[repeat_i,:,:] = NewInfectants
            cum_sum_a_vec.append(sum_a_vec)
            cum_k_vec.append(k_vec)
            cum_R0_vec.append(R0_vec)
            cumX.append(X)
            cumY.append(Y)
            cumN.append(N)
            cumType.append(Type)
            repeat_i = repeat_i+1
                   
    return(cumStatus, cumNewInfectants, cumInfectedFrom, cumType, cumX, cumY, cum_sum_a_vec, cum_k_vec, cumN)



###################### シミュレーション結果プロット  ####################
def plot_simulation(cumStatus, cumNewInfectants, cumInfectedFrom, cumX, cumY, cum_sum_a_vec, cum_k_vec, cumN, fontsize_main, title):
    
    #############
    # グラフ表示
    #############
    c1,c2,c3,c4 = "black","red","green","blue"
    numberE = np.array([np.sum(cumStatus[i]==1, axis=0) for i in range(n_repeat*n_sub_repeat)]).T
    numberI = np.array([np.sum(cumStatus[i]==2, axis=0) for i in range(n_repeat*n_sub_repeat)]).T
    numberR = np.array([np.sum(cumStatus[i]==3, axis=0) for i in range(n_repeat*n_sub_repeat)]).T
    sumE = np.sum(numberE,1)
    sumI = np.sum(numberI,1)
    sumR = np.sum(numberR,1)
    
    if show_daily_new_infectants:
        plt.figure(figsize=(16,12))
        plt.subplots_adjust(wspace=0.3, hspace=0.5, left=0.05, right=0.95, bottom=0.1, top=0.9)
        if n_repeat <= 12:
          plotrows = 3
          plotcols = 4
        else:
          plotrows = 4
          plotcols = 5
        for i in range(n_repeat):
            if i%(plotrows*plotcols)==0 and i!=0:
                pdf.savefig()
                plt.figure(figsize=(16,12))
                plt.subplots_adjust(wspace=0.3, hspace=0.5, left=0.05, right=0.95, bottom=0.1, top=0.9)
            plt.subplot(plotrows,plotcols,i%(plotrows*plotcols)+1)
            plt.bar(range(len(cumNewInfectants[i,:,0])), cumNewInfectants[i,:,0],color='b')
            plt.bar(range(len(cumNewInfectants[i,:,1])), cumNewInfectants[i,:,1], bottom=cumNewInfectants[i,:,0], color='r')
            plt.ylim([0,max_daily])
            plt.xticks(np.arange(0, t_end + 1, 30), fontsize=8)
            plt.yticks(fontsize=9)
            plt.xlabel('日', fontsize=12)
            plt.ylabel('人数', fontsize=12)
            if np.any(np.array(injected_trials)<0) or np.any(np.array(injected_trials)==i):
              plt.title('新規感染者数 (試行 '+str(i+1)+', 変異型 at '+str(t_new)+'day)', fontsize=10)
            else:
              plt.title('新規感染者数 (試行 '+str(i+1)+')', fontsize=10)
        pdf.savefig()

        plt.figure(figsize=(10,7))
        plt.subplot()
        plt.subplots_adjust(left=0.3, right=0.9, bottom=0.15, top=0.9)
        plt.bar(range(cumNewInfectants.shape[1]), np.sum(cumNewInfectants[:,:,0],0),color='b')
        plt.bar(range(cumNewInfectants.shape[1]), np.sum(cumNewInfectants[:,:,1],0), bottom=np.sum(cumNewInfectants[:,:,0],0), color='r')
        plt.ylim([0,max_dailysum])
        plt.xticks(np.arange(0, t_end + 1, 30), fontsize=24)
        plt.yticks(fontsize=24)
        plt.xlabel('日', fontsize=24, labelpad=15)
        plt.ylabel('人数', fontsize=24, labelpad=15)
        plt.title('新規感染者数 (合計)', fontsize=26)
        pdf.savefig()

        print('doubling_rate = '+ str(doubling_rate(np.sum(cumNewInfectants[:,30:60,0],0))))
             
    plt.figure(figsize=(10,7))

    # 未感染者を非表示
    ax1 = plt.subplot(111)
    plt.subplots_adjust(left=0.15, right=0.8, bottom=0.15, top=0.85)
    plt.plot(numberE, color=c2, lw=0.5);
    plt.plot(numberI, color=c3, lw=0.5);
    plt.plot(numberR, color=c4, lw=0.5);
    plt.xticks(np.arange(0, t_end + 1, 30), fontsize=24) # 14
    plt.yticks(fontsize=24) #18
    plt.xlabel("日",fontsize=24, labelpad=15) # 18
    plt.ylabel("個別波及人数",fontsize=24, labelpad=15) #18
    plt.axvline(t1, c='black', lw=0.5)
    plt.axvline(t2, c='black', lw=0.5)
    plt.title(title, fontsize=24, pad=20) # 18
    ax2 = ax1.twinx()
    ax2.plot(sumE, color=c2, label='Exposed', lw=3);
    ax2.plot(sumI, color=c3, label='Infectious', lw=3);
    ax2.plot(sumR, color=c4, label='Recovered', lw=3);
    ax1ylim = ax1.get_ylim()
    ax2.set_ylim(ax1ylim[0]*100,ax1ylim[1]*100)
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    ax2.tick_params(axis='y', labelsize=22)
    ax2.set_ylabel('合計人数',fontsize=24, labelpad=20)
    plt.legend(fontsize=20, loc="upper left") # 14
    
    pdf.savefig()
    pdf.close()

def doubling_rate(x0):
  n = len(x0)
  logx0 = np.log2(x0)
  meanx = np.mean(logx0)
  x = logx0-meanx
  t = np.arange(n)-(0+(n-1))/2
  return(np.sum(x*x) / np.sum(t*x))


In [ ]:

############################################
################  Main  ####################
############################################

for repeat_index in repeat_indices:

  task_sumR = []
  from scipy.stats import lognorm
  from scipy.stats import gamma
  import numpy as np
  from matplotlib import pyplot as plt

  if serial_interval_type == 'Nishiura':
    ws = lognorm.pdf(np.arange(ws_length+1),scale=4, s=np.log(1.8))
    ws[0]=0
    ws = ws/np.sum(ws)
    #print('ws=',ws)
  elif serial_interval_type == 'Bi':
    Bi_mean = 6.3
    Bi_sd = 4.2
    Bi_theta = Bi_sd**2/Bi_mean
    Bi_k = Bi_mean/Bi_theta
    ws=np.append(np.zeros(1),gamma.pdf(np.arange(1,ws_length+1),Bi_k,scale=Bi_theta))
    ws = ws/np.sum(ws)

  plt.figure(figsize=(15,6))
  plt.subplot(1,3,1)
  plt.subplots_adjust(wspace=0.3, left=0.125, right=0.95, bottom=0.175, top=0.8)
  plt.plot(np.arange(1,(ws_length+1)),ws[1:])
  plt.title('ws',fontsize=18)
  plt.ylabel("Probability",fontsize=18)
  plt.xlabel('Days',fontsize=18)
  plt.xticks(fontsize=18)
  plt.yticks(fontsize=18)
  plt.show()

  for taskID in taskID_range:

    plt.close('all')
    print('<< taskID = ',taskID,' >>')

    #############  commandline parameter  ##########

    parm = parms[taskID-1]
    exec(parms_str + '=' + str(parm) )
    print('< ',parms_str,'=',parm,' >')

    savetitle0 = savetitle_header+'_'+parms_str+'_'+str(parm)+'_'+str(repeat_index)
    savetitle = os.path.join(path, savetitle0)

    ###### for calculation of lognormal distribution #########
    # mean = exp(log(scale)+sigma**2/2 )=scale*exp(sigma**2/2)
    # scale = mean/exp(sigma**2/2)
    scale1 = (mean1)/math.exp(sigma1**2/2)
    kscale = kmean/math.exp(ksigma**2/2)
    rscale = rmean/math.exp(rsigma**2/2)
    pdf = PdfPages(savetitle+'.pdf') # save all figures for each task here

    # R0とkとrの分布のグラフ
    plt.figure(figsize=(15,6))

    Rmax=15
    kmax=30
    rmax=50

    plt.subplot(1,3,1)
    plt.subplots_adjust(wspace=0.425, left=0.125, right=0.95, bottom=0.175, top=0.8)
    # graph of R
    y1 = stats.lognorm.pdf(np.arange(0,Rmax+0.01,0.01), s=sigma1, scale=scale1)
    plt.plot(np.arange(0,Rmax+0.01,0.01),y1);
    plt.title("R0: 平均="+str(mean1)+", 分散="+str(sigma1),fontsize=18);
    plt.xlabel("R0",fontsize=18);
    plt.ylabel("割合",fontsize=18);
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    plt.subplot(1,3,2)
    # graph of k
    kcum = stats.lognorm.cdf(np.arange(0.5, kmax+1.5), s=ksigma, scale=kscale)
    k = np.concatenate((kcum[0].reshape(1,),kcum[1:]-kcum[:-1]))
    plt.plot(np.arange(kmax+1),k);
    plt.title("接触人数 k: 平均="+str(kmean)+", 分散="+str(ksigma),fontsize=18);
    plt.xlabel("k",fontsize=18);
    plt.ylabel("割合",fontsize=18);
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    plt.subplot(1,3,3)
    # graph of r
    rcum = stats.lognorm.cdf(np.arange(0.5, rmax+1.5), s=rsigma, scale=rscale)
    r = np.concatenate((rcum[0].reshape(1,),rcum[1:]-rcum[:-1]))
    plt.plot(np.arange(rmax+1),r);
    plt.title("行動半径 r: 平均="+str(rmean)+", 分散="+str(rsigma),fontsize=18);
    plt.xlabel("r",fontsize=18);
    plt.ylabel("割合",fontsize=18);
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    pdf.savefig()

    # シミュレーション実施
    cumStatus, cumNewInfectants, cumInfectedFrom, cumType, cumX, cumY, cum_sum_a_vec, cum_k_vec, cumN = simulate()
    # シミュレーション結果プロット
    plot_simulation(cumStatus, cumNewInfectants, cumInfectedFrom, cumX, cumY, cum_sum_a_vec, cum_k_vec, cumN, fontsize_main=18, \
                    title="各感染状態合計人数（太線：平均、細線：各試行）")

    # 結果保存
    if save_data:
      f = open(savetitle+'.pkl','wb')
      pickle.dump([cumStatus,cumNewInfectants,cumInfectedFrom,cumType,cumX,cumY,cum_sum_a_vec, cum_k_vec,cumN], f)
      f.close()

    # パラメータ記録
    f = open(savetitle+'_params.txt', 'w')
    vars = ['parms','parms_str','taskID','min_area_halfsize'  ,'b' ,'c' ,'t_end', 'n_repeat' ,'n_sub_repeat' ,'show_daily_new_infectants' ,'savetitle' ,'reduction1' ,'reduction2' ,'initial_E_number','t1' ,'t2' ,'t_new','growth_rate_factor' ,'sigma1' ,'mean1' ,'ksigma' ,'kmean','rsigma','rmean']
    for var in vars:
        exec('f.write(var+" = "+str('+var+'))')
        f.write('\n')
    f.close()

    # keep record of sumR 
    numberR = np.array([np.sum(cumStatus[i]==3, axis=0) for i in range(n_repeat*n_sub_repeat)]).T
    sumR = np.sum(numberR,1)
    task_sumR.append(sumR)

  # Meta figure
  savetitle_meta = os.path.join(path, savetitle_header+'_'+parms_str+'_'+str(repeat_index))
  pdf = PdfPages(savetitle_meta+'.pdf') # save meta figure
  plt.figure(figsize=(10,7))
  ax1 = plt.subplot(111)
  plt.subplots_adjust(left=0.15, right=0.8, bottom=0.15, top=0.85)
  for taski in range(len(taskID_range)):
    plt.plot(task_sumR[taski], lw=3);
  plt.xticks(np.arange(0, t_end + 1, 30), fontsize=24) # 14
  plt.yticks(fontsize=24) #18
  ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
  plt.xlabel("日",fontsize=24, labelpad=10) # 18
  plt.axvline(t1, c='black', lw=0.5)
  plt.axvline(t2, c='black', lw=0.5)
  plt.title('感染を経験した人の合計人数', fontsize=24, pad=20) # 18
  plt.ylabel('人数',fontsize=24,labelpad=15);
  plt.legend([parms_str+"="+str(parms[i-1]) for i in taskID_range], fontsize=16, loc="upper left"); # 14
  pdf.savefig()
  pdf.close()


  if save_data:
    f = open(savetitle_meta+'_task_cumR.pkl','wb')
    pickle.dump(task_sumR, f)
    f.close()


In [ ]:
############################################
######    アニメーションを作成    ##########
############################################

show_n_repeats = 5
full_area_view = True

taskID_range = list(range(1,len(parms)+1))

for taskID in taskID_range:

  print('<< taskID = ',taskID,' >>')

  parm = parms[taskID-1]
  exec(parms_str + '=' + str(parm) )
  print('< ',parms_str,'=',parm,' >')

  savetitle0 = savetitle_header+'_'+parms_str+'_'+str(parm)
  savetitle = os.path.join(path, savetitle0)

  # データ読み込み
  f = open(savetitle+'.pkl','rb')
  [cumStatus,cumNewInfectants,cumInfectedFrom,cumType,cumX,cumY,cum_sum_a_vec, cum_k_vec,cumN] = pickle.load(f)
  f.close()

  ###### for calculation of lognormal distribution #########
  # mean = exp(log(scale)+sigma**2/2 )=scale*exp(sigma**2/2)
  # scale = mean/exp(sigma**2/2)
  scale1 = (mean1)/math.exp(sigma1**2/2)
  kscale = kmean/math.exp(ksigma**2/2)

  #numR = np.sum(cumStatus[repeat_i]==3,0)
  settle_times_list = [np.min(np.where(np.sum(cumStatus[repeat_i]==3,0) == np.max(np.sum(cumStatus[repeat_i]==3,0)))[0]) for repeat_i in range(show_n_repeats)] # np.array(temp, dtype=int)
  settle_times = np.array(settle_times_list, dtype=int)
  #print(settle_times)
  each_length = settle_times + 1 + settle_length
  #print(each_length>t_end)
  each_length[each_length > t_end] = t_end
  each_length = each_length + title_length # video frame length for each repeat
  cum_length = np.cumsum(each_length) # cumulative frame length

  #### 表示する最大のエリアサイズnmaxと各人のa（感染確率；二次元配列）を計算、cum...に入れる。
  cum_nmax = [];
  cum_a_mat_all = [];
  for repeat_i in range(show_n_repeats):
    X = cumX[repeat_i]
    Y = cumY[repeat_i]
    sum_a_vec = cum_sum_a_vec[repeat_i]
    nmax = int(max(np.max(abs(X)), np.max(abs(Y)), int(min_area_halfsize)))
    nmax = min(nmax, max_area_halfsize)    #  at most, max_area_halfsize
    a_mat_all = np.array([prob_a() for i in range((2*nmax+1)**2)]).reshape(2*nmax+1,2*nmax+1)
    for i in range(len(sum_a_vec)):
      if abs(X[i])<=nmax and abs(Y[i])<=nmax:
        a_mat_all[int(X[i])+nmax,int(Y[i])+nmax] = sum_a_vec[i]
    cum_a_mat_all.append(a_mat_all)
    cum_nmax.append(nmax)
    #sum_a_vec_norm = sum_a_vec_all/a_max
    #sum_a_vec_norm[sum_a_vec_norm>1] = 1
    

  def plot_frame(t,repeat_i):
      
      Status = cumStatus[repeat_i]
      X = cumX[repeat_i] # 各感染者の位置
      Y = cumY[repeat_i]
      nmax = cum_nmax[repeat_i] # そのリピートでの最大エリアサイズ
      InfectedFrom = cumInfectedFrom[repeat_i]
      Type = cumType[repeat_i]
      n0 = int(max(np.max(abs(X[Status[:,t]>0])),np.max(abs(Y[Status[:,t]>0])),int(min_area_halfsize)))
      if not full_area_view:
        n0 = min(n0, nmax) # その時点での表示エリアサイズ
      mag = min(n0, nmax)
      #print(n0, end=' ' )
      cumR = Status==3
      sumR = np.sum(cumR,0)
      maxR = np.max(sumR)

      if not full_area_view or n0 <= nmax:
        a_mat_all = cum_a_mat_all[repeat_i]
        a_mat_t = a_mat_all[(nmax-n0):(nmax+n0+1),:]
        a_mat_t = a_mat_t[:,(nmax-n0):(nmax+n0+1)]
        if t<t1:
            a_mat_t = a_mat_t
        elif t<t2:
            a_mat_t = a_mat_t*reduction1
        else:
            a_mat_t = a_mat_t*reduction2
        a_mat_norm = a_mat_t/a_max
        a_mat_norm[a_mat_norm>1] = 1
        cmap = plt.get_cmap('YlOrRd')
        ec = cmap(a_mat_norm.flatten())
        xcoord = np.tile(np.arange(-n0,n0+1),(2*n0+1,1)).flatten()
        ycoord = np.tile(np.arange(-n0,n0+1),(2*n0+1,1)).T.flatten()
        plt.scatter(xcoord, ycoord, marker='.', c='white', edgecolors=ec, s=labeledgesize/mag, linewidths=linewidth/mag); #OK
      
      # Type=0 is old virus, 1 is new virus
      if full_area_view:
        validXY0 = np.array(Type==0,dtype=bool) #np.ones(X.shape[0],dtype=bool)
        validXY1 = np.array(Type==1,dtype=bool) #np.ones(X.shape[0],dtype=bool)
      else:
        validXY0 = np.logical_and.reduce((abs(X)<=n0,abs(Y)<=n0,Type==0))
        validXY1 = np.logical_and.reduce((abs(X)<=n0,abs(Y)<=n0,Type==1))

      # Type0
      # E
      Ex0 = X[np.logical_and(Status[:,t] == 1, validXY0)]
      Ey0 = Y[np.logical_and(Status[:,t] == 1, validXY0)]
      # I
      Ix0 = X[np.logical_and(Status[:,t] == 2, validXY0)]
      Iy0 = Y[np.logical_and(Status[:,t] == 2, validXY0)]
      # R
      Rx0 = X[np.logical_and(Status[:,t] == 3, validXY0)]
      Ry0 = Y[np.logical_and(Status[:,t] == 3, validXY0)]      
      # Type1
      # E
      Ex1 = X[np.logical_and(Status[:,t] == 1, validXY1)]
      Ey1 = Y[np.logical_and(Status[:,t] == 1, validXY1)]
      # I
      Ix1 = X[np.logical_and(Status[:,t] == 2, validXY1)]
      Iy1 = Y[np.logical_and(Status[:,t] == 2, validXY1)]
      # R
      Rx1 = X[np.logical_and(Status[:,t] == 3, validXY1)]
      Ry1 = Y[np.logical_and(Status[:,t] == 3, validXY1)]  

      #print(Ex.shape[0]+Ix.shape[0]+Rx.shape[0], end=';' )
      
      plt.scatter(Ex0, Ey0, marker=".", c='yellow', s=labelsize/mag, linewidths=0);
      plt.scatter(Ix0, Iy0, marker=".", c='magenta', s=labelsize/mag, linewidths=0);
      plt.scatter(Rx0, Ry0, marker=".", c='green', s=labelsize/mag, linewidths=0);
      plt.scatter(Ex1, Ey1, marker=".", c='yellow', s=labelsize/mag, linewidths=0);
      plt.scatter(Ix1, Iy1, marker=".", c='red', s=labelsize/mag, linewidths=0);
      plt.scatter(Rx1, Ry1, marker=".", c='blue', s=labelsize/mag, linewidths=0);
      plt.xlim([-n0*1.1, n0*1.1])
      plt.ylim([-n0*1.1, n0*1.1])
               
      #plt.scatter(xcoord.flatten(), ycoord.flatten(), c=c, s=labelsize/mag, linewidths=0);
      plt.title('Day '+str(t), fontsize=26)
      if t>0:
        newly_infected = np.where(np.logical_and(Status[:,t-1]==0, Status[:,t]==1))[0]
        if len(newly_infected)>0:
            for infected in newly_infected:
              if infected>=0:
                source = InfectedFrom[infected]
                ax = plt.gca()
                targetX = X[infected]
                targetY = Y[infected]
                if full_area_view:  # そのまま矢印を表示
                  ax.arrow(x=X[source], y=Y[source], dx=targetX-X[source], dy=targetY-Y[source], width=arrowwidth/mag, head_width=headwidth/mag, head_length=headlength/mag, length_includes_head=True,  facecolor='black')
                elif abs(X[source])<n0 and abs(Y[source])<n0: # エリア内に矢印を短縮して表示
                  shrink_X = max((targetX-X[source])/(n0*1.05-X[source]),(targetX-X[source])/(-n0*1.05-X[source]))
                  shrink_Y = max((targetY-Y[source])/(n0*1.05-Y[source]),(targetY-Y[source])/(-n0*1.05-Y[source]))
                  shrink = max(shrink_X, shrink_Y)
                  if shrink > 1:
                    targetX = X[source] + (targetX-X[source])/shrink
                    targetY = Y[source] + (targetY-Y[source])/shrink 
                  ax.arrow(x=X[source], y=Y[source], dx=targetX-X[source], dy=targetY-Y[source], width=arrowwidth/mag, head_width=headwidth/mag, head_length=headlength/mag, length_includes_head=True,  facecolor='black')


  def update(j):
      repeat_i = np.where(j < cum_length)[0][0]
      i = j - np.concatenate((np.zeros(1,dtype=int),cum_length))[repeat_i]
      if i==0:
          print('\nrepeat ' + str(repeat_i))
      print(str(i), end=' ')
      if (i+1)%40 == 0:
        print('')

      if j>0:
          plt.cla()
      if i < title_length:
          plt.gca().xaxis.set_visible(False)
          plt.gca().yaxis.set_visible(False)
          plt.gca().text(0.5,0.55,"試行 "+str(repeat_i+1), fontsize=50,horizontalalignment='center', verticalalignment='center');
      else:
          plt.gca().set_visible(True)
          plot_frame(i-title_length, repeat_i)


  def make_animation(show_n_repeats, title):
      fig = plt.figure(figsize=(img_w, img_h), dpi=resolution)
      startflag = True
      ani = anm.FuncAnimation(fig, update, interval=interval, frames=cum_length[show_n_repeats-1])
      ani.save(title, writer='pillow')
      print('\nfinished')



  make_animation(show_n_repeats, savetitle+'.gif')


  # パラメータ記録
  f = open(savetitle+'_anim_params.txt', 'w')
  vars = ['img_w' ,'img_h' ,'resolution','interval' ,'frames' ,'labeledgesize' ,'labelsize' ,'headwidth' ,'headlength' ,'a_max' ,'title_length' ,'settle_length' ,'show_n_repeats']
  for var in vars:
      exec('f.write(var+" = "+str('+var+'))')
      f.write('\n')
  f.close()
